# nesajov



In [21]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [22]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from PIL import Image
import torchvision.transforms as transforms

from torch import jit

import io
import time

import cv2


In [23]:
# Check device
use_cuda = torch.cuda.is_available()

#device = torch.device('cuda' if use_cuda else 'cpu')
device='cpu'

In [24]:
device

'cpu'

In [25]:
IMG_PATH = '..\images'
MODEL_PATH = ''

### Load model

In [29]:
    # Model   
    # Load ScriptModule from io.BytesIO object
    with open('model_traced-2.pt', 'rb') as f:
        buffer = io.BytesIO(f.read())
        
    traced_model = torch.jit.load(buffer, map_location=device)
    #print('[WARNING] ScriptModules cannot be moved to a GPU device yet. Running strictly on CPU for now.')
    #device = torch.device('cpu') # 'to' is not supported on TracedModules (yet)

In [30]:
output = traced_model(torch.ones(1, 3, 224, 224))

In [31]:
output[0, :5]

tensor([ 4.3491, -5.0439], grad_fn=<SliceBackward>)

### Predict image

In [11]:
#vimage = cv2.imread(IMG_PATH + '/Birman_46-224.jpg')
#vimage = cv2.imread(IMG_PATH + '/leonberger_35-224.jpg')
#vimage = cv2.imread(IMG_PATH + '/C12_12-512.jpg')
vimage = cv2.imread(IMG_PATH + '/D6_16-512.jpg')

In [10]:
x = vimage[:, :, (2, 1, 0)]
x = torch.from_numpy(x).permute(2, 0, 1).to(device)
to_norm_tensor = transforms.Compose([
        #transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
img_tensor = to_norm_tensor(x.float().div_(255))
#print('Image tensor: {}'.format(img_tensor))
print('Image tensor shape: {}'.format(img_tensor.shape))
img_tensor.unsqueeze_(0) # add a dimension for the batch
print('New shape: {}'.format(img_tensor.shape))
outputs = traced_model(img_tensor)
score, predicted = outputs.max(1)
print(f'Predicted: {predicted.item()} | {score.item()}')

Image tensor: tensor([[[-2.0995, -2.0995, -2.0995,  ..., -2.0801, -2.0608, -2.0608],
         [-2.0995, -2.0995, -2.0995,  ..., -2.0995, -2.1189, -2.0995],
         [-2.0995, -2.0995, -2.0995,  ..., -2.0801, -2.0801, -2.0801],
         ...,
         [-2.1771, -2.2158, -2.3903,  ..., -2.3515, -2.3515, -2.3709],
         [-2.2546, -2.2546, -2.2158,  ..., -2.3515, -2.3515, -2.3709],
         [-2.2546, -2.2546, -2.2352,  ..., -2.3515, -2.3709, -2.3903]],

        [[-2.0839, -2.0839, -2.0839,  ..., -2.0643, -2.0446, -2.0446],
         [-2.0839, -2.0839, -2.0839,  ..., -2.0839, -2.1036, -2.0839],
         [-2.0839, -2.0839, -2.0839,  ..., -2.0643, -2.0643, -2.0643],
         ...,
         [-2.1626, -2.2019, -2.3789,  ..., -2.3396, -2.3396, -2.3593],
         [-2.2413, -2.2413, -2.2019,  ..., -2.3396, -2.3396, -2.3593],
         [-2.2413, -2.2413, -2.2216,  ..., -2.3396, -2.3593, -2.3789]],

        [[-1.8897, -1.8897, -1.8897,  ..., -1.8702, -1.8507, -1.8507],
         [-1.8897, -1.8897, -1.